<지스트신문>에서 활동하면서 기사 주제를 찾아야 하는데 매일 뉴스에 들어가는 것이 어려워 가끔 좋은 인터뷰를 놓치는 경우가 많다. 또한 다양한 기사를 통해서 좋은 기사 거리를 찾을 수 있다. 이에, 기사의 헤드라인을 크롤링하는 프로그램을 만들어 기사거리를 쉽게 받고자 한다. 사용자의 편의를 위해 2가지 함수를 통해 정보를 받을 예정이다. 첫번 째는 관심사를 검색하여 크롤링하는 프로그램을 이용하고, 두번 째는 헤드라인들 크롤링하는 프로그램을 제작할 예정이다.

Step 1 필요한 모듈로 부터 필요한 함수 임포트하기

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import datetime #시간 연산 라이브러리 
import time
import sys        #system 설정을 변경하기 위해 필요
import os
import numpy           
import pandas  as pd 
import math  #페이지 번호 계산을 위해
import urllib.request
import urllib.parse

Step 2.저장할 경로 및  파일명 셋팅하기 

In [2]:
#현재 작업 디렉토리 확인

print(os.getcwd())

c:\ai


In [3]:
now = time.strftime('%Y-%m-%d-%I-%M-%S') #프로그램을 돌리는 시점 대한민국 시각을 now에 저장 
print(now)

directory = 'c:/ai/DATA/Clothes'
os.makedirs(directory)
os.chdir(directory)
print(os.getcwd())

2020-11-04-08-16-05
c:\ai\DATA\2020-11-04-08-16-05 기준 스크랩 폴더


In [4]:
cnt = int(input('1.크롤링할 데이터 개수를 쓰세요 : '))
real_cnt = math.ceil(cnt / 10)  #페이지를 내릴 수 (10개 마다 갱신 됨) 
f_name = input('1.txt 형태로 저장할 파일명을 확장자 포함해서 쓰세요:(예: article.txt) :')
fc_name = input('2.csv 형태로 저장할 파일명을 확장자 포함해서 쓰세요:(예: article.csv) :')
fx_name = input('3.xlsx 형태로 저장할 파일명을 확장자 포함해서 쓰세요:(예: article.xlsx) :')

1.크롤링할 데이터 개수를 쓰세요 : 50
1.txt 형태로 저장할 파일명을 확장자 포함해서 쓰세요:(예: article.txt) :article_headline.txt
2.csv 형태로 저장할 파일명을 확장자 포함해서 쓰세요:(예: article.csv) :article_headline.csv
3.xlsx 형태로 저장할 파일명을 확장자 포함해서 쓰세요:(예: article.xlsx) :article_headline.xlsx


Step 3 크롬 드라이버로 웹 브라우저 실행

In [5]:
s_time = time.time( )
options = Options()
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
driver = webdriver.Chrome(options = options, executable_path=r'c:/Temp/chromedriver_240/chromedriver.exe')

#구글 뉴스 홈페이지에 들어간다.
driver.get('https://news.google.com/topstories?hl=ko&gl=KR&ceid=KR:ko')

Step 4  검색 유무 확인 전 검색 크롤링 함수 및 헤드라인 크롤링 함수 선언

4-1. 구글 뉴스는 페이지 넘김이 없고 스크롤 다운으로만 새로운 데이터가 생성됨 그러므로 스크롤 다운 함수를 이용해야함.

In [6]:
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(1)

전역 변수 지정, 둘 중 하나의 함수만 실행되므로 전역 변수에는 겹치는 값이 할당될 일이 없다.

In [7]:
no2=[]                  # 기사 번호 컬럼
title2=[ ]              # 기사 제목 컬럼
press2=[ ]           # 기사 작성 언론사 컬럼
article_time2=[ ]       # 기사 작성 시간 컬럼
addr2=[ ]               # 기사 주소 컬럼
n=1

Step 5 검색어가 없을 때

In [8]:

def article_headline():
    #구글 뉴스의 헤드라인 접속
    driver.find_element_by_xpath("""//*[@id="yDmH0d"]/c-wiz/div/div[2]/div[2]/div/main/c-wiz/div[1]/div[1]/div[2]/span/a""").click()
    time.sleep(1)
    
    t = 1 
    #스크롤 다운 
    while (t<=real_cnt):
        scroll_down(driver)
        t+=1


    no = 1           # 기사 번호 초기값
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')



    notice_list1 = soup.select('h3 > a ') # 게시글 제목 및 url을 얻는 데 사용
    notice_list = notice_list1[:cnt]

    for i in notice_list:
        f = open(f_name, 'a',encoding='UTF-8')
        notice_list = soup.select('h3 > a ') # 게시글 제목 및 url을 얻는 데 사용
        # 1. 게시글 번호
        no2.append(no)
        f.write("1.번호: " + str(no) + "\n")
        print("1.번호: %s" %no)
        
        # 2. 게시글 제목 
        title = i.string
        title2.append(title)
        f.write('2.제목: '+str(title)+'\n')
        print("2.제목:",title)
            
        notice_list = soup.select('h3 > a ') # 게시글 제목 및 url을 얻는 데 사용
        addr = 'https://news.google.com' + i.get('href')[1:]
        addr2.append(addr)
        f.write('3.기사 주소: '+str(addr)+'\n')
        print("3.기사 주소:",addr)                           
        while no<=cnt :
            notice_list = soup.select('article > div.QmrVtf.RD0gLb.kybdz > div ') #기사 제목 과 js path가 달라 재설정
            i = notice_list[no]
            #기사 작성 언론사
            press = i.find('a').get_text()
            press2.append(press)
            f.write('4.언론사: '+str(press)+'\n')
            print("4.언론사:",press)        
                    
            #5 기사 시간 
            article_time = i.find('time').get_text()
            article_time2.append(str(now)+" 기준 "+article_time)
            f.write('5.기사 작성 일시: '+str(now)+" 기준 " + str(article_time)+'\n')
            print("5.기사 작성 일시:"+str(now)+" 기준" ,article_time) 
            print("\n")
            no +=1
            break  
    driver.close( ) 


Step 6 검색어가 있을 때

In [9]:
def article_search():
    #구글 뉴스에서 검색어 입력 
    element = driver.find_element_by_xpath("""//*[@id="gb"]/div[2]/div[2]/div/form/div[1]/div/div/div/div/div[1]/input[2]""")
    element.send_keys(query_txt)
    time.sleep(2)
    driver.find_element_by_xpath("""//*[@id="gb"]/div[2]/div[2]/div/form/button[4]""").click()
    
    #스크롤 다운 초기 값
    t = 1 
    #스크롤 다운 
    while (t<=real_cnt):
        scroll_down(driver)
        t+=1    
    
    #검색 버튼 누름

    no = 1           # 기사 번호 초기값
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')



    notice_list1 = soup.select('div > article > h3 > a ') # 게시글 제목 및 url을 얻는 데 사용
    notice_list = notice_list1[14:cnt+14] 
    #알수 없는 이유로 이전 페이지에서 14개의 항목을 크롤링하여 임의로 수정하여 검색값만 나오도록 수정함 

    for i in notice_list:
        f = open(f_name, 'a',encoding='UTF-8')
        notice_list = soup.select('h3 > a ') # 게시글 제목 및 url을 얻는 데 사용
        # 1. 게시글 번호
        no2.append(no)
        f.write("1.번호: " + str(no) + "\n")
        print("1.번호: %s" %no)
        
        # 2. 게시글 제목 
        title = i.string
        title2.append(title)
        f.write('2.제목: '+str(title)+'\n')
        print("2.제목:",title)
            
        notice_list = soup.select('h3 > a ') # 게시글 제목 및 url을 얻는 데 사용
        addr = 'https://news.google.com' + i.get('href')[1:]
        addr2.append(addr)
        f.write('3.기사 주소: '+str(addr)+'\n')
        print("3.기사 주소:",addr)                           
        while no<=cnt :
            notice_list = soup.select('article > div.QmrVtf.RD0gLb.kybdz > div ') #기사 제목 과 js path가 달라 재설정
            i = notice_list[no]
            #기사 작성 언론사
            press = i.find('a').get_text()
            press2.append(press)
            f.write('4.언론사: '+str(press)+'\n')
            print("4.언론사:",press)        
                    
            #5 기사 시간 
            article_time = i.find('time').get_text()
            article_time2.append(str(now)+" 기준 "+article_time)
            f.write('5.기사 작성 일시: '+str(now)+" 기준 " + str(article_time)+'\n')
            print("5.기사 작성 일시:"+str(now)+" 기준" ,article_time) 
            print("\n")
            no +=1
            break  
    
    driver.close( ) 


In [10]:
#검색 여부 확인
while 1:
    query_choice = input('기사에서 검색할 필요한 검색어가 있습니까? (y/n)')
    if query_choice == 'y':
        query_txt= input("검색할 검색어를 입력하세요: ")
        print(article_search()) #검색을 하였을 때의 함수를 이용 
        break
    elif query_choice =='n':
        print("검색어가 없으니 헤드라인을 출력합니다.")
        print(article_headline()) #검색 안하였을 때의 함수를 이용
        break
    else :
        print('y 또는 n을 입력하세요') #y혹은 n만 입력하도록 설정 '

기사에서 검색할 필요한 검색어가 있습니까? (y/n)n
검색어가 없으니 헤드라인을 출력합니다.
1.번호: 1
2.제목: 미 대선 개표 지연은 선거인단과 우편투표 등 주마다 다른 복잡한 제도 탓
3.기사 주소: https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3LmNob3N1bi5jb20vaW50ZXJuYXRpb25hbC91cy8yMDIwLzExLzA0L0dQT0pIWkhVUEZHSDNGSFlHNjZLM1k0RzRNL9IBXWh0dHBzOi8vd3d3LmNob3N1bi5jb20vaW50ZXJuYXRpb25hbC91cy8yMDIwLzExLzA0L0dQT0pIWkhVUEZHSDNGSFlHNjZLM1k0RzRNLz9vdXRwdXRUeXBlPWFtcA?hl=ko&gl=KR&ceid=KR%3Ako
4.언론사: YTN news
5.기사 작성 일시:2020-11-04-08-16-05 기준 1시간 전


1.번호: 2
2.제목: 2020 미국대선 플로리다주의 투표소가 102%의 투표율을 기록한 이유
3.기사 주소: https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3Lmh1ZmZpbmd0b25wb3N0LmtyL2VudHJ5L2Zsb3JpZGEtdm90ZS1wcmVzaWRlbnRzX2tyXzVmYTI2ZDczYzViNjEyOGM2YjVlNDM0ZtIBWmh0dHBzOi8vbS5odWZmaW5ndG9ucG9zdC5rci9hbXAvZW50cnkvZmxvcmlkYS12b3RlLXByZXNpZGVudHNfa3JfNWZhMjZkNzNjNWI2MTI4YzZiNWU0MzRmLw?hl=ko&gl=KR&ceid=KR%3Ako
4.언론사: 한겨레
5.기사 작성 일시:2020-11-04-08-16-05 기준 7시간 전


1.번호: 3
2.제목: 제네시스 몰며 청년주택 혜택 꿀꺽…서울시 적발
3.기사 주소: https://news.google.com/articles/CAIiEHuzr7TOhgr23EGVhHHED

4.언론사: 동아일보
5.기사 작성 일시:2020-11-04-08-16-05 기준 10시간 전


1.번호: 27
2.제목: 검찰, ‘술접대 의혹’ 김봉현 부른 날 박훈 “검사 3명 이름 안다” - 중앙일보
3.기사 주소: https://news.google.com/articles/CBMiRGh0dHBzOi8vbW5ld3Muam9pbnMuY29tL2FydGljbGUvMjM5MTE3OTU_Y2xvYz1qb29uZ2FuZy1taG9tZS1Hcm91cDIy0gEraHR0cHM6Ly9tbmV3cy5qb2lucy5jb20vYW1wYXJ0aWNsZS8yMzkxMTc5NQ?hl=ko&gl=KR&ceid=KR%3Ako
4.언론사: 중앙일보
5.기사 작성 일시:2020-11-04-08-16-05 기준 6시간 전


1.번호: 28
2.제목: [생중계] 국회 운영위 청와대 국정감사 (2020.11.4 오후)
3.기사 주소: https://news.google.com/articles/CCAiC0J3Tk9IblQzWVYwmAEB?hl=ko&gl=KR&ceid=KR%3Ako
4.언론사: YTN
5.기사 작성 일시:2020-11-04-08-16-05 기준 6시간 전


1.번호: 29
2.제목: 주식 양도세 부과 기준 10억으로 유지
3.기사 주소: https://news.google.com/articles/CBMiRWh0dHBzOi8vd3d3LmNob3N1bi5jb20vZWNvbm9teS8yMDIwLzExLzA0L05OUUJERkU0UjVEQTNIQUpDNUhGWFBHWElBL9IBVGh0dHBzOi8vd3d3LmNob3N1bi5jb20vZWNvbm9teS8yMDIwLzExLzA0L05OUUJERkU0UjVEQTNIQUpDNUhGWFBHWElBLz9vdXRwdXRUeXBlPWFtcA?hl=ko&gl=KR&ceid=KR%3Ako
4.언론사: 매일경제
5.기사 작성 일시:2020-11-04-08-16-05 기준 9시간 전


1.번호: 30
2.제목: 전당원투표와 여론조사 사이…스텝 꼬

Step 7 pandas를 활용하여 데이터 프레임 생성 

In [11]:
#pandas 활용하여 데이터 프레임 생성
google_article= pd.DataFrame()
google_article['번호']=no2
google_article['제목']=title2
google_article['기사 주소']=addr2
google_article['언론사']=press2
google_article['기사 작성 일시']=article_time2
google_article

,번호,제목,기사 주소,언론사,기사 작성 일시
0,1,미 대선 개표 지연은 선거인단과 우편투표 등 주마다 다른 복잡한 제도 탓,https://news.google.com/articles/CBMiTmh0dHBzO...,YTN news,2020-11-04-08-16-05 기준 1시간 전
1,2,2020 미국대선 플로리다주의 투표소가 102%의 투표율을 기록한 이유,https://news.google.com/articles/CBMiV2h0dHBzO...,한겨레,2020-11-04-08-16-05 기준 7시간 전
2,3,제네시스 몰며 청년주택 혜택 꿀꺽…서울시 적발,https://news.google.com/articles/CAIiEHuzr7TOh...,조선일보,2020-11-04-08-16-05 기준 5시간 전
3,4,합참 “동부 전선서 북한 남성 1명 신병 확보”,https://news.google.com/articles/CBMiRGh0dHBzO...,동아일보,2020-11-04-08-16-05 기준 5시간 전
4,5,홍남기 “인사권자 뜻대로 부총리로서 최선 다하겠다”,https://news.google.com/articles/CBMiPmh0dHA6L...,허프포스트코리아,2020-11-04-08-16-05 기준 1시간 전
5,6,"“사랑하는 북녘 동포 여러분!” 이인영, 판문점서 외쳐",https://news.google.com/articles/CBMiRmh0dHBzO...,JTBC News,2020-11-04-08-16-05 기준 6시간 전
6,7,“그랜저 몰면서 청년주택 NO”…차량가액 2468만원으로 제한 - 중앙일보,https://news.google.com/articles/CBMiJ2h0dHBzO...,노컷뉴스,2020-11-04-08-16-05 기준 12시간 전
7,8,"北 민간인, 최전방 철책 넘어 귀순…징후 포착하고도 36시간 속수무책",https://news.google.com/articles/CBMiRGh0dHBzO...,YTN news,2020-11-04-08-16-05 기준 2시간 전
8,9,김종인 “야권연대? 야권이 국민의힘 말고 더 있나”,https://news.google.com/articles/CBMiV2h0dHBzO...,뉴스TVCHOSUN,2020-11-04-08-16-05 기준 5시간 전
9,10,"“사랑하는 북녘 동포 여러분!” 이인영, 판문점서 외쳐",https://news.google.com/articles/CBMiRmh0dHBzO...,매일경제,2020-11-04-08-16-05 기준 9시간 전


In [12]:
#csv 형태로 저장하기
google_article.to_csv(fc_name,encoding="utf-8-sig",index=False)

In [13]:
#엑셀 형태로 저장하기
google_article.to_excel(fx_name,index=False)

In [14]:
#크롤링 결과 요약해서 출력하기
e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *70)

print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %f_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xlsx 파일명 : %s " %fx_name)
print("=" *70)





총 소요시간은 59.7 초 입니다 
파일 저장 완료: txt 파일명 : article_headline.txt 
파일 저장 완료: csv 파일명 : article_headline.csv 
파일 저장 완료: xlsx 파일명 : article_headline.xlsx 
